# LIS640 Assignment 3: Adversarial Attack

## Requirements

In assignment 3, your task is to implement adversarial attack on a trained deep learnig model for Fashion-MNIST Classification. Model training process has been included in this file. Please run the following modular `Get the Model` to get the model checkpoint `LeNet.pt`.

After getting the model, you need to fill the missing code in the modular `Adversarial Attack` to finish the following two tasks:

1. Implement a non-targeted white-box Fast Gradient Sign Method (FGSM) attack (https://arxiv.org/abs/1412.6572) against the deep learning model `LeNet.pt`.

2. Implement a non-targeted white-box Basic Iterative Method (BIM) attack (https://arxiv.org/abs/1611.01236) against the deep learning model `LeNet.pt`.


**Note: You should get a low evaluation accuracy after performing the adversarial attack.**

# Get the Model

## Remember to Use GPU in Your Colab

In [1]:
!nvidia-smi

Wed Dec 13 17:10:08 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   51C    P8              20W /  80W |   1213MiB /  6144MiB |     16%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Set the Seed

In [2]:
import torch
import random
import numpy as np
def setup_seed(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True

setup_seed(42)

## Load Dataset

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split

# Load the dataset and train, val, test splits
print("Loading datasets...")
FASHION_transform = transforms.Compose([
    transforms.ToTensor(), # Transform from [0,255] uint8 to [0,1] float
])
FASHION_trainval = datasets.FashionMNIST('.', download=True, train=True, transform=FASHION_transform)
FASHION_train = Subset(FASHION_trainval, range(50000))
FASHION_val = Subset(FASHION_trainval, range(50000,60000))
FASHION_test = datasets.FashionMNIST('.', download=True, train=False, transform=FASHION_transform)
print("Done!")

trainloader = DataLoader(FASHION_train, batch_size=128, shuffle=True)
valloader = DataLoader(FASHION_val, batch_size=128, shuffle=True)
testloader = DataLoader(FASHION_test, batch_size=128, shuffle=True)


Loading datasets...


100%|██████████| 26421880/26421880 [00:02<00:00, 9313145.83it/s] 


Extracting .\FashionMNIST\raw\train-images-idx3-ubyte.gz to .\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 243997.15it/s]


Extracting .\FashionMNIST\raw\train-labels-idx1-ubyte.gz to .\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4464381.76it/s]


Extracting .\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to .\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting .\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to .\FashionMNIST\raw

Done!


## Define the Network

In [4]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 4 * 4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    self.dropout = nn.Dropout(0.1)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 4 * 4)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

## Train the Network

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
model = Network().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=5e-5)
num_epoch = 20

def train(model, trainloader, valloader, num_epoch): # Train the model
  print("Start training...")
  model.train() # Set the model to training mode
  loss_train = []
  loss_val = []
  for i in range(num_epoch):
    running_loss = []
    for batch, label in tqdm(trainloader):
      batch = batch.to(device)
      label = label.to(device)
      optimizer.zero_grad() # Clear gradients from the previous iteration
      pred = model(batch) # This will call Network.forward() that you implement
      loss = criterion(pred, label) # Calculate the loss
      running_loss.append(loss.item())
      loss.backward() # Backprop gradients to all tensors in the network
      optimizer.step() # Update trainable weights
    print("Epoch {} loss:{}".format(i+1,np.mean(running_loss))) # Print the average loss for this epoch
    loss_train.append(np.mean(running_loss))

    running_loss_val = []
    model.eval() # Set the model to evaluation mode
    for batch, label in tqdm(valloader):
      batch = batch.to(device)
      label = label.to(device)
      pred = model(batch) # This will call Network.forward() that you implement
      loss = criterion(pred, label) # Calculate the loss
      running_loss_val.append(loss.item())
    print("Epoch {} val_loss:{}".format(i+1,np.mean(running_loss_val))) # Print the average loss for this epoch
    loss_val.append(np.mean(running_loss_val))

  print("Done!")
  return loss_train, loss_val

def evaluate(model, loader): # Evaluate accuracy on validation / test set
  model.eval() # Set the model to evaluation mode
  correct = 0
  with torch.no_grad(): # Do not calculate grident to speed up computation
    for batch, label in tqdm(loader):
      batch = batch.to(device)
      label = label.to(device)
      pred = model(batch)
      correct += (torch.argmax(pred,dim=1)==label).sum().item()
  acc = correct/len(loader.dataset)
  print("Evaluation accuracy: {}".format(acc))
  return acc

loss_train, loss_val = train(model, trainloader, valloader, num_epoch)
print("Evaluate on validation set...")
evaluate(model, valloader)
print("Evaluate on test set")
evaluate(model, testloader)

torch.save(model,'LeNet.pt')

Start training...


100%|██████████| 391/391 [00:10<00:00, 38.74it/s] 


Epoch 1 loss:0.8342842438336834


100%|██████████| 79/79 [00:00<00:00, 140.09it/s]


Epoch 1 val_loss:0.6443771316280847


...

Epoch 20 val_loss:0.28993322068377386
Done!
Evaluate on validation set...


100%|██████████| 79/79 [00:00<00:00, 137.96it/s]


Evaluation accuracy: 0.8959
Evaluate on test set


100%|██████████| 79/79 [00:00<00:00, 144.99it/s]

Evaluation accuracy: 0.8899


# Adversarial Attack


## Task 1: FGSM

In [6]:
def fgsm(x, y, model, eps):
  x.requires_grad = True


  output = model(x)
  model.zero_grad()

  loss = nn.CrossEntropyLoss()(output, y)
  loss.backward() 

  data_grad = x.grad.data

  sign_data_grad = data_grad.sign()
  x_adv = x + eps * sign_data_grad
  
  x_adv = torch.clamp(x_adv, 0, 1)

  return x_adv

def fgsm_test(model, loader, eps):
  model.to(device).eval()
  correct = 0.
  for x, y in tqdm(loader):
    x, y = x.to(device), y.to(device)
    x_adv = fgsm(x, y, model, eps)
    pred = model(x_adv)
    correct += (torch.argmax(pred,dim=1)==y).sum().item()
  acc = correct / len(loader.dataset)
  print("Evaluation accuracy: {}".format(acc))
  return acc

model = torch.load('LeNet.pt')
fgsm_test(model, testloader, eps=25/255)

100%|██████████| 79/79 [00:00<00:00, 82.17it/s] 

Evaluation accuracy: 0.0665


0.0665

## Task 2: BIM

In [7]:
def bim(x, y, model, eps, iters, alpha):
# Make a copy of the input
  x_adv = x.clone().detach().requires_grad_(True).to(device)

  for i in range(iters):
      # Forward pass
      output = model(x_adv)
      model.zero_grad()

      # Calculate the loss
      loss = nn.CrossEntropyLoss()(output, y)
      loss.backward()  # Compute gradients

      # Apply perturbation 
      x_adv = x_adv + alpha * x_adv.grad.sign()

      x_adv = torch.min(torch.max(x_adv, x - eps), x + eps) 
      x_adv = torch.clamp(x_adv, 0, 1).detach_().requires_grad_(True)

  return x_adv


def bim_test(model, loader, eps, iters, alpha):
  model.to(device).eval()
  correct = 0.
  for x, y in tqdm(loader):
    x, y = x.to(device), y.to(device)
    x_adv = bim(x, y, model, eps, iters, alpha)
    pred = model(x_adv)
    correct += (torch.argmax(pred,dim=1)==y).sum().item()
  acc = correct / len(loader.dataset)
  print("Evaluation accuracy: {}".format(acc))
  return acc

model = torch.load('LeNet.pt')
bim_test(model, testloader, eps=25/255, iters=10, alpha=3/255)

100%|██████████| 79/79 [00:01<00:00, 42.98it/s]

Evaluation accuracy: 0.0053


0.0053